In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from torch.utils.data import DataLoader, Dataset
import numpy as np 
import pandas as pd
import os
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GroupKFold
import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
from tqdm import tqdm, tqdm_notebook
from scipy.stats import spearmanr
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
import pickle
from apex import amp
import random
import shutil
import transformers
from spacy.lang.en import English
from spacy_readability import Readability
from math import floor, ceil
#from comment_parser import comment_parser
import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
from xml.sax.saxutils import unescape
import torch.nn.init as init
tqdm.pandas()

In [3]:
os.listdir('./input')

['train_10_fold.csv', 'train_new_features.csv', 'train_5_fold.csv']

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [5]:
SEED                = 2020
DATA_DIR            ="./input/"
WORK_DIR            = "./working/"
nlp                 = English()
device              = torch.device('cuda')
seed_everything(SEED)
epochs              = 4
lr                  = 3e-5
batch_size          = 8
accumulation_steps  = 1
NUM_FOLD            = 10
warmup              = 0.04
config = transformers.BertConfig.from_pretrained('bert-base-uncased',output_hidden_states=True)  

In [6]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
train_df = pd.read_csv(os.path.join(DATA_DIR,"train_new_features.csv"))
print('loaded %d train records' % len(train_df))

loaded 6079 train records


In [7]:
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    first_sep = True
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _trim_input(title, question, answer, max_sequence_length, 
                t_max_len=30, q_max_len=239, a_max_len=239):

    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (t_new_len+a_new_len+q_new_len+4)))
        
        t = t[:t_new_len]
        q_len_head = round(q_new_len/3)
        q_len_tail = -1* (q_new_len -q_len_head)
        a_len_head = round(a_new_len/3)
        a_len_tail = -1* (a_new_len -a_len_head) 
        
        q = q[:q_len_head]+q[q_len_tail:]
        a = a[:a_len_head]+a[a_len_tail:]
    
    return t, q, a

def convert_lines(title, question, answer, max_sequence_length, tokenizer, t_max_len_seq=30, q_max_len_seq=239, a_max_len_seq=239):
    
    all_tokens   = []
    all_masks    = []
    all_segments = []
    
    longer = 0
    
    for t, q, a in tqdm(zip(title, question, answer)):
        
        tokens_t, tokens_q, tokens_a  = _trim_input(t, q, a, max_sequence_length=max_sequence_length)
        #print(tokens_t)
        #print(tokens_q)
        #print(tokens_a)
        
        stoken = ["[CLS]"] + tokens_t + ["[SEP]"] + tokens_q + ["[SEP]"] + tokens_a + ["[SEP]"] 
        ##############
        #token_ids
        ##############
        token_ids = tokenizer.convert_tokens_to_ids(stoken)
        input_ids = token_ids + [0] * (max_sequence_length-len(token_ids))
        
        #############
        #input_masks
        #############
        attention_masks = _get_masks(stoken, max_sequence_length)
        #print(attention_masks)
        
        ##############
        #input_segments
        ###############
        input_segments = _get_segments(stoken, max_sequence_length)
        
        all_tokens.append(input_ids)
        all_masks.append(attention_masks)
        all_segments.append(input_segments)
        #break
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [8]:
train_tokens, train_masks, train_segments = convert_lines(train_df["question_title"],
                                                          train_df["question_body"], 
                                                          train_df["answer"],
                                                          max_sequence_length=512, 
                                                          tokenizer=tokenizer)

6079it [00:38, 157.21it/s]


## External Features

In [9]:
def add_external_features(df):
    
    eng_stopwords = set(stopwords.words("english"))
    #If the question is longer, it may be more clear, which may help users give a more 
    df['question_body']      = df['question_body'].progress_apply(lambda x:str(x))
    df['question_num_words'] = df.question_body.str.count('\S+')
    df['question_title_num_words'] = df.question_title.str.count('\S+')
    
    #The assumption here is that longer answer could bring more useful detail
    df['answer']            = df['answer'].progress_apply(lambda x:str(x))
    df['answer_num_words']  = df.answer.str.count('\S+')
    
    df["question_title_num_unique_words"] = df["question_title"].progress_apply(lambda x: len(set(str(x).split())))
    df["question_body_num_unique_words"]  = df["question_body"].progress_apply(lambda x: len(set(str(x).split())))
    df["answer_num_unique_words"]         = df["answer"].progress_apply(lambda x: len(set(str(x).split())))
    
    df["question_title_num_chars"] = df["question_title"].apply(lambda x: len(str(x)))
    df["question_body_num_chars"]  = df["question_body"].apply(lambda x: len(str(x)))
    df["answer_num_chars"]         = df["answer"].apply(lambda x: len(str(x)))
    
    df['qt_words'] = df['question_title'].apply(lambda s: [f for f in s.split() if f not in eng_stopwords])
    df['q_words'] = df['question_body'].apply(lambda s: [f for f in s.split() if f not in eng_stopwords])
    df['a_words'] = df['answer'].apply(lambda s: [f for f in s.split() if f not in eng_stopwords])
    df['qa_word_overlap'] = df.apply(lambda s: len(np.intersect1d(s['q_words'], s['a_words'])), axis = 1)
    df['qt_word_overlap'] = df.apply(lambda s: len(np.intersect1d(s['qt_words'], s['a_words'])), axis = 1)
    
    df['qa_word_overlap_norm'] = df.apply(lambda s: s['qa_word_overlap']/(len(s['a_words']) + len(s['q_words'])  - s['qa_word_overlap']) , axis = 1)
    df['qta_word_overlap_norm'] = df.apply(lambda s: s['qt_word_overlap']/(len(s['a_words']) + len(s['qt_words']) - s['qt_word_overlap']), axis = 1)
    df.drop(['q_words', 'a_words', 'qt_words'], axis = 1, inplace = True)
    
    return df

In [10]:
train_df = add_external_features(train_df)

100%|██████████| 6079/6079 [00:00<00:00, 47679.84it/s]


In [11]:
handmade_cols = ["question_body_num_unique_words", 'question_num_words',
                 "question_title_num_unique_words", "question_title_num_words", 
                 "answer_num_unique_words", "answer_num_words",
                ]

num_words_scaler = MinMaxScaler()
num_words_scaler.fit(train_df[handmade_cols].values)
with open('scaler.pickle', mode='wb') as f:
    pickle.dump(num_words_scaler, f)

train_df[handmade_cols]=  num_words_scaler.transform(train_df[handmade_cols].values)
train_handmade_features= train_df[handmade_cols+['l2_qt_a', 'l2_qb_a', 'cos_qt_a', 'cos_qb_a',
                                                'qa_word_overlap_norm',
                                                'qta_word_overlap_norm']].values

MinMaxScaler(copy=True, feature_range=(0, 1))

In [12]:
def label_encoder(x, dict_reverse):
    try:
        return dict_reverse[x]
    except:
        return 0

unique_categories = train_df['category'].unique().tolist()
category_dict = {i + 1: e for i, e in enumerate(unique_categories)}
category_dict_reverse = {v: k for k, v in category_dict.items()}
with open('category.pickle', mode='wb') as f:
    pickle.dump(category_dict_reverse, f)
train_df['category'] = train_df['category'].apply(lambda x: label_encoder(x, category_dict_reverse))

unique_hosts = train_df['host'].unique().tolist()
host_dict = {i + 1: e for i, e in enumerate(unique_hosts)}
host_dict_reverse = {v: k for k, v in host_dict.items()}
with open('host.pickle', mode='wb') as f:
    pickle.dump(host_dict_reverse, f)
train_df['host'] = train_df['host'].apply(lambda x: label_encoder(x, host_dict_reverse))

n_cat    = len(category_dict_reverse) + 1
cat_emb  = 128
n_host   = len(host_dict_reverse) + 1
host_emb = 128

In [13]:
class QuestDataset(Dataset):

    def __init__(self, df, token_ids, masks, segments, hosts, categories, handmade_features):
                
        self.df         = df
        self.token_ids  = token_ids
        self.masks      = masks
        self.segments   = segments
        self.hosts      = hosts
        self.categories = categories
        self.handmades  = handmade_features

        self.question_cols = ['question_asker_intent_understanding',
                              'question_body_critical', 'question_conversational',
                              'question_expect_short_answer', 'question_fact_seeking',
                              'question_has_commonly_accepted_answer',
                              'question_interestingness_others', 'question_interestingness_self',
                              'question_multi_intent', 'question_not_really_a_question',
                              'question_opinion_seeking', 'question_type_choice',
                              'question_type_compare', 'question_type_consequence',
                              'question_type_definition', 'question_type_entity',
                              'question_type_instructions', 'question_type_procedure',
                              'question_type_reason_explanation', 'question_type_spelling',
                              'question_well_written']
        self.answer_cols   = ['answer_helpful', 'answer_level_of_information',
                              'answer_plausible', 'answer_relevance',
                              'answer_satisfaction', 'answer_type_instructions',
                              'answer_type_procedure', 'answer_type_reason_explanation',
                              'answer_well_written']

        self.label = self.df[self.question_cols + self.answer_cols].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        token_id = self.token_ids[idx]
        mask     = self.masks[idx]
        segment  = self.segments[idx]
        host     = self.hosts[idx]
        category = self.categories[idx]
        handmade = self.handmades[idx]
        
        labels = self.label[idx]

        return [token_id, mask, segment, host, category, handmade], labels

In [14]:
def children(m):
    return m if isinstance(m, (list, tuple)) else list(m.children())

def set_trainable_attr(m, b):
    m.trainable = b
    for p in m.parameters():
        p.requires_grad = b


def apply_leaf(m, f):
    c = children(m)
    if isinstance(m, torch.nn.Module):
        f(m)
    if len(c) > 0:
        for l in c:
            apply_leaf(l, f)

def set_trainable(l, b):
    apply_leaf(l, lambda m: set_trainable_attr(m, b))

#  Model

In [15]:
class QuestModel6(nn.Module):

    def __init__(self, n_cat, cat_emb, n_host, host_emb, num_labels):
        super().__init__()
        
        BERT_DIMS = 768
        self.bert_model = transformers.BertModel.from_pretrained('bert-base-uncased', config=config)    
        set_trainable(self.bert_model.embeddings.word_embeddings, False)
        
        self.category_embedding = nn.Embedding(n_cat, cat_emb)
        self.host_embedding     = nn.Embedding(n_host, host_emb)
        
        self.fc1  = nn.Linear(BERT_DIMS*4, BERT_DIMS*4)
        self.fc2  = nn.Linear(BERT_DIMS*4 + int(cat_emb) + int(host_emb) + 12, 21)
        self.fc3  = nn.Linear(BERT_DIMS*4 + int(cat_emb) + int(host_emb) + 12, 9)
        self.tanh = nn.SELU()
        
        self._init_weights(self.category_embedding)
        self._init_weights(self.host_embedding)
        self._init_weights(self.fc1)
        self._init_weights(self.fc2)
        self._init_weights(self.fc3)
        
    def _init_weights(self, module):
        """ Initialize the weights """
        if isinstance(module, (nn.Linear, nn.Embedding)):
            print("initailize weight")
            module.weight.data.normal_(mean=0.0, std=0.02)
        if isinstance(module, nn.Linear) and module.bias is not None:
            print("initailize bias")
            module.bias.data.zero_()

        
    def forward(self, token_ids, masks, segments, hosts, categories, handmades):
        
        category_embed      = self.category_embedding(categories)
        host_embed          = self.host_embedding(hosts)
        
        external_features   = torch.cat((category_embed, host_embed, handmades), 1)
        _,_, hidden_layers = self.bert_model(input_ids=token_ids, 
                                             token_type_ids=segments, 
                                             attention_mask=masks)
        
        hidden_input  = [hidden_layers[-1][:, 0, :],
                         hidden_layers[-2][:, 0, :],
                         hidden_layers[-3][:, 0, :], 
                         hidden_layers[-4][:, 0, :]]
        
        cls           = torch.cat(hidden_input, dim = -1)
        cls           = self.fc1(cls)
        cls           = self.tanh(cls)
        cls           = torch.cat((cls, external_features), 1)
        q_results     = self.fc2(cls)
        a_results     = self.fc3(cls)
        
        results       = torch.cat((q_results, a_results), 1)
        
        return results

In [16]:
def train_model(model, criterion, train_loader, optimizer, scheduler):
    max_grad_norm = 1.0
    model.train()
    avg_loss = 0.
    question_weight = 0.7
    answer_weight = 0.3
    
    optimizer.zero_grad()
    for idx, (inputs, labels) in enumerate(train_loader):
        token_ids, masks, segments, hosts, categories, handmades = inputs
        token_ids   = token_ids.long().cuda()
        masks       = masks.long().cuda()
        segments    = segments.long().cuda()
        hosts       = hosts.long().cuda()
        categories  = categories.long().cuda()
        handmades   = handmades.float().cuda()
        
        labels = labels.float().cuda()
        output_train = model(token_ids, masks, segments, hosts, categories, handmades)
        loss = question_weight*criterion(output_train[:,0:21], labels[:,0:21]) + \
               answer_weight*criterion(output_train[:,21:30], labels[:,21:30])
        
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), max_grad_norm)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        
        
    return avg_loss

def val_model(model, criterion, val_loader):
    avg_val_loss = 0.
    model.eval()
    preds    = []
    original = []
    question_weight = 0.7
    answer_weight = 0.3
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(val_loader):
            token_ids, masks, segments, hosts, categories, handmades = inputs
            token_ids   = token_ids.long().cuda()
            masks       = masks.long().cuda()
            segments    = segments.long().cuda()
            hosts       = hosts.long().cuda()
            categories  = categories.long().cuda()
            handmades   = handmades.float().cuda()
            labels      = labels.float().cuda()
            
            output_val = model(token_ids, masks, segments, hosts, categories, handmades)
            loss = question_weight*criterion(output_val[:,0:21], labels[:,0:21]) + \
                   answer_weight*criterion(output_val[:,21:30], labels[:,21:30])
            avg_val_loss += loss.item() / len(val_loader)
            preds.append(torch.sigmoid(output_val).cpu().numpy())
            original.append(labels.cpu().numpy())
        
        trues = np.concatenate(original)
        preds = np.concatenate(preds)
        
        rhos = []
        for col_trues, col_pred in zip(trues.T, preds.T):
            rhos.append(np.nan_to_num(spearmanr(col_trues, col_pred + np.random.normal(0, 1e-7, col_pred.shape[0])).correlation))
        
    return avg_val_loss, np.mean(rhos)

# CV Strategy

In [17]:
from sklearn.utils import check_random_state
from sklearn.utils.validation import _num_samples, check_array
from sklearn.utils.multiclass import type_of_target

from sklearn.model_selection._split import _BaseKFold, _RepeatedSplits, \
    BaseShuffleSplit, _validate_shuffle_split

In [18]:
targets = [
        'question_asker_intent_understanding',
        'question_body_critical',
        'question_conversational',
        'question_expect_short_answer',
        'question_fact_seeking',
        'question_has_commonly_accepted_answer',
        'question_interestingness_others',
        'question_interestingness_self',
        'question_multi_intent',
        'question_not_really_a_question',
        'question_opinion_seeking',
        'question_type_choice',
        'question_type_compare',
        'question_type_consequence',
        'question_type_definition',
        'question_type_entity',
        'question_type_instructions',
        'question_type_procedure',
        'question_type_reason_explanation',
        'question_type_spelling',
        'question_well_written',
        'answer_helpful',
        'answer_level_of_information',
        'answer_plausible',
        'answer_relevance',
        'answer_satisfaction',
        'answer_type_instructions',
        'answer_type_procedure',
        'answer_type_reason_explanation',
        'answer_well_written'    
    ] + ['host', 'category']

In [19]:
#https://www.kaggle.com/ratthachat/ml-stratifiers
def IterativeStratification(labels, r, random_state):
    """This function implements the Iterative Stratification algorithm described
    in the following paper:
    Sechidis K., Tsoumakas G., Vlahavas I. (2011) On the Stratification of
    Multi-Label Data. In: Gunopulos D., Hofmann T., Malerba D., Vazirgiannis M.
    (eds) Machine Learning and Knowledge Discovery in Databases. ECML PKDD
    2011. Lecture Notes in Computer Science, vol 6913. Springer, Berlin,
    Heidelberg.
    """

    n_samples = labels.shape[0]
    test_folds = np.zeros(n_samples, dtype=int)

    # Calculate the desired number of examples at each subset
    c_folds = r * n_samples

    # Calculate the desired number of examples of each label at each subset
    c_folds_labels = np.outer(r, labels.sum(axis=0))

    labels_not_processed_mask = np.ones(n_samples, dtype=bool)

    while np.any(labels_not_processed_mask):
        # Find the label with the fewest (but at least one) remaining examples,
        # breaking ties randomly
        num_labels = labels[labels_not_processed_mask].sum(axis=0)

        # Handle case where only all-zero labels are left by distributing
        # across all folds as evenly as possible (not in original algorithm but
        # mentioned in the text). (By handling this case separately, some
        # code redundancy is introduced; however, this approach allows for
        # decreased execution time when there are a relatively large number
        # of all-zero labels.)
        if num_labels.sum() == 0:
            sample_idxs = np.where(labels_not_processed_mask)[0]

            for sample_idx in sample_idxs:
                fold_idx = np.where(c_folds == c_folds.max())[0]

                if fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(fold_idx.shape[0])]

                test_folds[sample_idx] = fold_idx
                c_folds[fold_idx] -= 1

            break

        label_idx = np.where(num_labels == num_labels[np.nonzero(num_labels)].min())[0]
        if label_idx.shape[0] > 1:
            label_idx = label_idx[random_state.choice(label_idx.shape[0])]

        sample_idxs = np.where(np.logical_and(labels[:, label_idx].flatten(), labels_not_processed_mask))[0]

        for sample_idx in sample_idxs:
            # Find the subset(s) with the largest number of desired examples
            # for this label, breaking ties by considering the largest number
            # of desired examples, breaking further ties randomly
            label_folds = c_folds_labels[:, label_idx]
            fold_idx = np.where(label_folds == label_folds.max())[0]

            if fold_idx.shape[0] > 1:
                temp_fold_idx = np.where(c_folds[fold_idx] ==
                                         c_folds[fold_idx].max())[0]
                fold_idx = fold_idx[temp_fold_idx]

                if temp_fold_idx.shape[0] > 1:
                    fold_idx = fold_idx[random_state.choice(temp_fold_idx.shape[0])]

            test_folds[sample_idx] = fold_idx
            labels_not_processed_mask[sample_idx] = False

            # Update desired number of examples
            c_folds_labels[fold_idx, labels[sample_idx]] -= 1
            c_folds[fold_idx] -= 1

    return test_folds


class MultilabelStratifiedKFold(_BaseKFold):
    """Multilabel stratified K-Folds cross-validator
    Provides train/test indices to split multilabel data into train/test sets.
    This cross-validation object is a variation of KFold that returns
    stratified folds for multilabel data. The folds are made by preserving
    the percentage of samples for each label.
    Parameters
    ----------
    n_splits : int, default=3
        Number of folds. Must be at least 2.
    shuffle : boolean, optional
        Whether to shuffle each stratification of the data before splitting
        into batches.
    random_state : int, RandomState instance or None, optional, default=None
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedKFold that only uses random_state
        when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> mskf = MultilabelStratifiedKFold(n_splits=2, random_state=0)
    >>> mskf.get_n_splits(X, y)
    2
    >>> print(mskf)  # doctest: +NORMALIZE_WHITESPACE
    MultilabelStratifiedKFold(n_splits=2, random_state=0, shuffle=False)
    >>> for train_index, test_index in mskf.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    Notes
    -----
    Train and test sizes may be slightly different in each fold.
    See also
    --------
    RepeatedMultilabelStratifiedKFold: Repeats Multilabel Stratified K-Fold
    n times.
    """

    def __init__(self, n_splits=3, shuffle=False, random_state=None):
        super(MultilabelStratifiedKFold, self).__init__(n_splits, shuffle, random_state)

    def _make_test_folds(self, X, y):
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(type_of_target_y))

        num_samples = y.shape[0]

        rng = check_random_state(self.random_state)
        indices = np.arange(num_samples)

        if self.shuffle:
            rng.shuffle(indices)
            y = y[indices]

        r = np.asarray([1 / self.n_splits] * self.n_splits)

        test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

        return test_folds[np.argsort(indices)]

    def _iter_test_masks(self, X=None, y=None, groups=None):
        test_folds = self._make_test_folds(X, y)
        for i in range(self.n_splits):
            yield test_folds == i

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedKFold, self).split(X, y, groups)


class RepeatedMultilabelStratifiedKFold(_RepeatedSplits):
    """Repeated Multilabel Stratified K-Fold cross validator.
    Repeats Mulilabel Stratified K-Fold n times with different randomization
    in each repetition.
    Parameters
    ----------
    n_splits : int, default=5
        Number of folds. Must be at least 2.
    n_repeats : int, default=10
        Number of times cross-validator needs to be repeated.
    random_state : None, int or RandomState, default=None
        Random state to be used to generate random state for each
        repetition as well as randomly breaking ties within the iterative
        stratification algorithm.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import RepeatedMultilabelStratifiedKFold
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> rmskf = RepeatedMultilabelStratifiedKFold(n_splits=2, n_repeats=2,
    ...     random_state=0)
    >>> for train_index, test_index in rmskf.split(X, y):
    ...     print("TRAIN:", train_index, "TEST:", test_index)
    ...     X_train, X_test = X[train_index], X[test_index]
    ...     y_train, y_test = y[train_index], y[test_index]
    ...
    TRAIN: [0 3 4 6] TEST: [1 2 5 7]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    TRAIN: [0 1 4 5] TEST: [2 3 6 7]
    TRAIN: [2 3 6 7] TEST: [0 1 4 5]
    See also
    --------
    RepeatedStratifiedKFold: Repeats (Non-multilabel) Stratified K-Fold
    n times.
    """
    def __init__(self, n_splits=5, n_repeats=10, random_state=None):
        super(RepeatedMultilabelStratifiedKFold, self).__init__(
            MultilabelStratifiedKFold, n_repeats, random_state,
            n_splits=n_splits)


class MultilabelStratifiedShuffleSplit(BaseShuffleSplit):
    """Multilabel Stratified ShuffleSplit cross-validator
    Provides train/test indices to split data into train/test sets.
    This cross-validation object is a merge of MultilabelStratifiedKFold and
    ShuffleSplit, which returns stratified randomized folds for multilabel
    data. The folds are made by preserving the percentage of each label.
    Note: like the ShuffleSplit strategy, multilabel stratified random splits
    do not guarantee that all folds will be different, although this is
    still very likely for sizeable datasets.
    Parameters
    ----------
    n_splits : int, default 10
        Number of re-shuffling & splitting iterations.
    test_size : float, int, None, optional
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        complement of the train size. By default, the value is set to 0.1.
        The default will change in version 0.21. It will remain 0.1 only
        if ``train_size`` is unspecified, otherwise it will complement
        the specified ``train_size``.
    train_size : float, int, or None, default is None
        If float, should be between 0.0 and 1.0 and represent the
        proportion of the dataset to include in the train split. If
        int, represents the absolute number of train samples. If None,
        the value is automatically set to the complement of the test size.
    random_state : int, RandomState instance or None, optional (default=None)
        If int, random_state is the seed used by the random number generator;
        If RandomState instance, random_state is the random number generator;
        If None, the random number generator is the RandomState instance used
        by `np.random`. Unlike StratifiedShuffleSplit that only uses
        random_state when ``shuffle`` == True, this multilabel implementation
        always uses the random_state since the iterative stratification
        algorithm breaks ties randomly.
    Examples
    --------
    >>> from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
    >>> import numpy as np
    >>> X = np.array([[1,2], [3,4], [1,2], [3,4], [1,2], [3,4], [1,2], [3,4]])
    >>> y = np.array([[0,0], [0,0], [0,1], [0,1], [1,1], [1,1], [1,0], [1,0]])
    >>> msss = MultilabelStratifiedShuffleSplit(n_splits=3, test_size=0.5,
    ...    random_state=0)
    >>> msss.get_n_splits(X, y)
    3
    >>> print(mss)       # doctest: +ELLIPSIS
    MultilabelStratifiedShuffleSplit(n_splits=3, random_state=0, test_size=0.5,
                                     train_size=None)
    >>> for train_index, test_index in msss.split(X, y):
    ...    print("TRAIN:", train_index, "TEST:", test_index)
    ...    X_train, X_test = X[train_index], X[test_index]
    ...    y_train, y_test = y[train_index], y[test_index]
    TRAIN: [1 2 5 7] TEST: [0 3 4 6]
    TRAIN: [2 3 6 7] TEST: [0 1 4 5]
    TRAIN: [1 2 5 6] TEST: [0 3 4 7]
    Notes
    -----
    Train and test sizes may be slightly different from desired due to the
    preference of stratification over perfectly sized folds.
    """

    def __init__(self, n_splits=10, test_size="default", train_size=None,
                 random_state=None):
        super(MultilabelStratifiedShuffleSplit, self).__init__(
            n_splits, test_size, train_size, random_state)

    def _iter_indices(self, X, y, groups=None):
        n_samples = _num_samples(X)
        y = check_array(y, ensure_2d=False, dtype=None)
        y = np.asarray(y, dtype=bool)
        type_of_target_y = type_of_target(y)

        if type_of_target_y != 'multilabel-indicator':
            raise ValueError(
                'Supported target type is: multilabel-indicator. Got {!r} instead.'.format(
                    type_of_target_y))

        n_train, n_test = _validate_shuffle_split(n_samples, self.test_size,
                                                  self.train_size)

        n_samples = y.shape[0]
        rng = check_random_state(self.random_state)
        y_orig = y.copy()

        r = np.array([n_train, n_test]) / (n_train + n_test)

        for _ in range(self.n_splits):
            indices = np.arange(n_samples)
            rng.shuffle(indices)
            y = y_orig[indices]

            test_folds = IterativeStratification(labels=y, r=r, random_state=rng)

            test_idx = test_folds[np.argsort(indices)] == 1
            test = np.where(test_idx)[0]
            train = np.where(~test_idx)[0]

            yield train, test

    def split(self, X, y, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like, shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
            Note that providing ``y`` is sufficient to generate the splits and
            hence ``np.zeros(n_samples)`` may be used as a placeholder for
            ``X`` instead of actual training data.
        y : array-like, shape (n_samples, n_labels)
            The target variable for supervised learning problems.
            Multilabel stratification is done based on the y labels.
        groups : object
            Always ignored, exists for compatibility.
        Returns
        -------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        Notes
        -----
        Randomized CV splitters may return different results for each call of
        split. You can make the results identical by setting ``random_state``
        to an integer.
        """
        y = check_array(y, ensure_2d=False, dtype=None)
        return super(MultilabelStratifiedShuffleSplit, self).split(X, y, groups)

## Training 

In [20]:
y_train = train_df[targets].values
X_train = np.array(range(len(train_df))).reshape(-1, 1)
kf = MultilabelStratifiedKFold(n_splits = NUM_FOLD, shuffle=True, random_state = SEED).split(X_train,y_train)
if os.path.exists('./input/train_{}_fold.csv'.format(NUM_FOLD)) == False:
    train_df['{}_fold'.format(NUM_FOLD)] = 0
    for fold, (train_index, val_index) in enumerate(kf):
        train_df.loc[val_index, '{}_fold'.format(NUM_FOLD)] = fold
    train_df.to_csv('./input/train_{}_fold.csv'.format(NUM_FOLD), index=False)

In [21]:
def train_model_kfold():
    
    fold_df = pd.read_csv('./input/train_{}_fold.csv'.format(NUM_FOLD))
    
    for fold in range(NUM_FOLD):
        print("fold:", fold)
        if fold < 7: 
            continue
        train_index = fold_df[fold_df['{}_fold'.format(NUM_FOLD)] != fold].index
        val_index = fold_df[fold_df['{}_fold'.format(NUM_FOLD)] == fold].index

        train_df_ = train_df.iloc[train_index]
        val_df    = train_df.iloc[val_index]
    
        train_set    = QuestDataset(train_df_,
                                    train_tokens[train_index], 
                                    train_masks[train_index], 
                                    train_segments[train_index],
                                    train_df['host'].values[train_index],
                                    train_df['category'].values[train_index],
                                    train_handmade_features[train_index],
                                    )
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
    
        val_set      = QuestDataset(val_df,
                                    train_tokens[val_index],
                                    train_masks[val_index],
                                    train_segments[val_index],
                                    train_df['host'].values[val_index],
                                    train_df['category'].values[val_index],
                                    train_handmade_features[val_index]
                                   )
        val_loader   = DataLoader(val_set, batch_size=batch_size, shuffle=False)
    

        model = QuestModel6(n_cat, cat_emb, n_host, host_emb, num_labels=30)
        model.to(device)
        
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
        optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        
        
        num_training_steps = int(epochs*len(train_df_)/batch_size/accumulation_steps)
        num_warmup_steps   = int(warmup * num_training_steps)
    
        best_avg_loss    = 100.0
        best_score       = 0.0
        best_param_loss  = None
        best_param_score = None
        i = 0
        optimizer        = transformers.AdamW(optimizer_grouped_parameters, lr=lr, correct_bias=False) 
        criterion        = nn.BCEWithLogitsLoss()
        scheduler        = transformers.get_cosine_schedule_with_warmup(optimizer, 
                                                                        num_warmup_steps=num_warmup_steps, 
                                                                        num_training_steps=num_training_steps)  
        model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)

    
        for epoch in range(epochs):
        
            if i == 5: break
            start_time   = time.time()
            avg_loss     = train_model(model, criterion, train_loader, optimizer, scheduler)
            avg_val_loss, score = val_model(model, criterion, val_loader)
            elapsed_time = time.time() - start_time 
            print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t score={:.6f} \t time={:.2f}s'.format(epoch + 1,
                                                                                                         epochs, 
                                                                                                         avg_loss, 
                                                                                                         avg_val_loss,
                                                                                                         score, 
                                                                                                         elapsed_time))
    
            if best_avg_loss > avg_val_loss:
                i = 0
                best_avg_loss = avg_val_loss 
                best_param_loss = model.state_dict()
            if best_score < score:
                best_score = score
                best_param_score = model.state_dict()
            else:
                i += 1
                
        torch.save(best_param_score, 'weight_score_best_{}.pt'.format(fold))
        
        torch.cuda.empty_cache()
        del train_df_
        del val_df
        del model
        gc.collect()

In [22]:
train_model_kfold()

fold: 0
fold: 1
fold: 2
fold: 3
fold: 4
fold: 5
fold: 6
fold: 7
initailize weight
initailize weight
initailize weight
initailize bias
initailize weight
initailize bias
initailize weight
initailize bias
Epoch 1/4 	 loss=0.3882 	 val_loss=0.3664 	 score=0.392814 	 time=128.36s
Epoch 2/4 	 loss=0.3555 	 val_loss=0.3618 	 score=0.402260 	 time=126.64s
Epoch 3/4 	 loss=0.3375 	 val_loss=0.3641 	 score=0.402916 	 time=129.00s
Epoch 4/4 	 loss=0.3250 	 val_loss=0.3664 	 score=0.400165 	 time=129.72s
fold: 8
initailize weight
initailize weight
initailize weight
initailize bias
initailize weight
initailize bias
initailize weight
initailize bias
Epoch 1/4 	 loss=0.3894 	 val_loss=0.3702 	 score=0.404522 	 time=138.13s
Epoch 2/4 	 loss=0.3544 	 val_loss=0.3618 	 score=0.420205 	 time=136.16s
Epoch 3/4 	 loss=0.3368 	 val_loss=0.3628 	 score=0.418234 	 time=135.98s
Epoch 4/4 	 loss=0.3241 	 val_loss=0.3663 	 score=0.414469 	 time=136.45s
fold: 9
initailize weight
initailize weight
initailize weigh